In [1]:
#Imports
import json
import numpy as np
import pandas as pd
import csv
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
def jasonReader(path):
    """ Reads the .jason generated with jasonGenartor the given path"""
    with open(path,'r') as miarch:
        loaded_dict = json.loads(miarch.read())
    return loaded_dict

In [3]:
def wholecellParser(dictres):
    #extract all the slices data
    allSlices = dictres['image']
    #seting columns names
    column_names = ['max_peaks_positions','max_peaks_intensities','min_peaks_positions','min_peaks_intensities','amplitudes','times_to_peaks']
    #define a dataframe
    df_sum = pd.DataFrame(columns=column_names)
    #populating the dataframe
    for i in column_names:
        df_sum.loc[:,i] = allSlices[i][:2]

    return df_sum

### Precisión y exactitud amplitud

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/jsons/'
file_tabulado = pd.read_excel(path + 'tab_prueba2.xlsx')
del file_tabulado['Unnamed: 0']

In [24]:
df_manual = pd.read_csv("/media/ana/My Passport/luis/analisis/DI_manual.csv", sep=",")
df_manual.head(2)

,experimento,foto,pico,Intensidad_max,tiempo_max,Intensidad_min,tiempo_min,Amplitud,TTP
0,200121,C1a000,1,43.19,125,17.89,118,1.414198,7
1,200121,C1a000,2,42.64,360,16.80,354,1.538095,6


In [43]:
path = '/media/ana/My Passport/luis/analisis/'
experiments = os.listdir(path + 'jsons/')
dif_amp = pd.DataFrame()
dias = df_manual['experimento'].drop_duplicates()
foto = str(df_manual['foto'].drop_duplicates).lower()
for j in dias:
    for k in foto:
        for i in range(0,len (experiments)):
            if (str(dias[j]) in experiments[i]) & (k in experiments[i]):
                try:
                    experiment = jasonReader(path + 'jsons/' + experiments[i])
                    df_file = wholecellParser(experiment)
                    df_file['file'] = experiments[i]
                    dif_amp = dif_amp.append(df_file, ignore_index=True)
                except:
                    print(experiments[i])

KeyError: 200121

In [34]:
dif_amp

,max_peaks_positions,max_peaks_intensities,min_peaks_positions,min_peaks_intensities,amplitudes,times_to_peaks,file
0,124,29,63,13,1.230769,61.0,200121_analysis_result_c1a000.json
1,358,29,302,13,1.230769,56.0,200121_analysis_result_c1a000.json
2,29,71,7,28,1.535714,22.0,200121_analysis_result_c1b000.json
3,107,70,86,28,1.500000,21.0,200121_analysis_result_c1b000.json
4,27,80,8,37,1.162162,19.0,200121_analysis_result_c1c000.json
5,86,79,66,37,1.135135,20.0,200121_analysis_result_c1c000.json
6,23,95,5,53,0.792453,18.0,200121_analysis_result_c1d000.json
7,70,95,51,54,0.759259,19.0,200121_analysis_result_c1d000.json
8,24,43,0,20,1.150000,24.0,200121_analysis_result_c2a000.json
9,257,42,235,19,1.210526,22.0,200121_analysis_result_c2a000.json


### DICRI

In [ ]:
tab = pd.read_csv('/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/seleccion/tabulado_imagenes.csv', encoding='utf-8')
del tab['Unnamed: 0']
pd.set_option('display.max_rows', None)
controles = tab[tab['tratamiento'] == 'c']
controles = controles.reset_index(drop=True)

In [ ]:
exps = os.listdir(path + 'jsons/')
experiments = []
for j in range(0,len(exps)):
    for i in range(0,len(controles)):
        if str(controles['experimento'][i]) in exps[j]:
            if str(controles['foto'][i]) in exps[j]:
                experiments.append(exps[j])

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/'
# experiments = os.listdir(path + 'jsons/')
complete_file = pd.DataFrame()
for i in range(0,len (experiments)):
    try:
        experiment = jasonReader(path + 'jsons/' + experiments[i])
        amp_files = slicesParser_amplitudes(experiment).filter(['transient1','transient2'])
        amp_files = amp_files[:][0:33]
        transient_max = []
        for j in range(1,len(amp_files)+1):
            transient_max.append(max(amp_files['transient1'][j],amp_files['transient2'][j]))
        amp_files['transient_max'] = transient_max
        amp_files.loc[len(amp_files)+1] = (amp_files.mean())
        amp_files.loc[len(amp_files)+1] = [amp_files['transient1'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],amp_files['transient2'][len(amp_files)]*100/amp_files['transient_max'][len(amp_files)],'']
        amp_files[str(experiments[i])] = list(range(1,len(amp_files)-1))+['promedios']+['porcentaje_max']
        complete_file = pd.concat([complete_file,amp_files], axis=1)
    except ValueError:
        print(experiments[i])

In [ ]:
complete_file.to_excel(path + 'DICRI_controles.xlsx', engine='xlsxwriter')  

### Amplitud de transitorio global vs DI

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/json_nuevos/'
file_tabulado = pd.read_csv(path + 'tabulado_parceado.csv')
file_tabulado.head() 

In [ ]:
experiment = jasonReader(path + 'jsons/050320c8_analysis_result_8b000.json')
allSlices = experiment['slices']
for i in range(0,len(allSlices)):
    print(allSlices[i]['amplitudes'])

In [ ]:
path = '/media/leandro/Volumen1TB/Lean/Analizador_imagenes_calcio/Luis/analisis/json_nuevos/whole_cell/'
experiments = os.listdir(path)
df = pd.DataFrame()
for experiment in experiments:
    df = pd.read_csv(path + experiment, sep = '\t', index_col=0).loc['amplitudes']
    for imagen in range(0,len(file_tabulado)):
        cel = file_tabulado['célula'].loc[imagen]
        foto = file_tabulado['foto'].loc[imagen]
        if (cel in experiment) & (foto in experiment):
            print(experiment,df)

#     print (experiment, df)

### ALBUMINA

In [ ]:
# Python code demonstrate creating  
# DataFrame from dict narray / lists  
# By default addresses. 
  
import pandas as pd 
  
# intialise data of lists. 
data = {'BSA':[102, 97, 79], 
        'HSA':[65, 74, 69.5],
        'RabSA':[57, 63.3, 70],
        'RSA':[50, 51, 52],
        'PSA':[45, 48, 55]} 
  
# Create DataFrame 
df = pd.DataFrame(data)
df_mean = df.mean()
df_std = df.std()
# Print the output. 
df_std

In [ ]:
import matplotlib.pyplot as plt

plt.errorbar(df_mean.index, df_mean, yerr=df_std, xerr=None, fmt='o')
plt.ylabel('% convertion', fontdict=None, labelpad=None,loc=None)
plt.savefig('/media/leandro/Volumen1TB/Lean/Albumina/%alb_convertion.png')

In [ ]:
cambios_fq = pd.read_csv('/media/leandro/Volumen1TB/Lean/Albumina/cambios_fq_completos.csv', sep= ' ')

In [ ]:
cambios_fq.loc()